# YouTube Data API v3 Examples

In [1]:
from youtube.search import *
from youtube.videos import *
from youtube.comment_threads import *
from youtube.pipelines import *
from yt_api_key import key

## Search API

In [2]:
query = 'FTX'
search_params = {
    'key': key,
    'part': 'snippet',
    'type': 'video',
    'maxResults': 50,
    'relevanceLanguage': 'en',
    'publishedAfter': '2022-12-01' + 'T00:00:00Z',
    'publishedBefore': '2022-12-01' + 'T23:59:59Z',
    'order': 'viewCount'
}

In [3]:
search_results = await fetch_search_results(query, search_params)

l_video_ids = list(set([i['id']['videoId'] for i in search_results]))
print(f"Fetched {len(l_video_ids)} Video IDs")

Fetched 14 Video IDs


## Videos API

In [4]:
video_params = {
    'key': key,
    'part': 'id,statistics,topicDetails'
}

In [5]:
# Concurrent
video_concurrent_results = await fetch_videos(l_video_ids, video_params) # Also works for 1 video ID as string

print(f"Fetched data for {len(l_video_ids)} videos")
print("Video ID    | Comments")
print("------------|------------")
for k,v in video_concurrent_results.items():
    print(f"{k} | {v['statistics']['commentCount']}")

Fetched data for 14 videos
Video ID    | Comments
------------|------------
7QEbjRKbrjY | 2112
f26kRRKrlj8 | 17
vdv05uU1m98 | 138
Zc57tP7ODNQ | 179
CG1bpu2KqeQ | 7
RDVXI84KSY8 | 126
PamZCTjqR0k | 27
IyoGdwVIwWw | 0
ipcpgND2-AY | 2
Bn0HOI9Qx0o | 52
8vwv23XYcrU | 13
0Hxf4Vf54PI | 4456
tC_orukfkGE | 27
HAnFA2FAl3o | 0


## CommentThreads API

In [6]:
comments_params = {
    'key': key,
    'part': 'id,replies,snippet',
    'order': 'time'
}

# Remove videos with less than n comments
n_comments_minimum = 0
l_video_ids = [v['id'] for k,v in video_concurrent_results.items() if v['statistics']['commentCount'] >= n_comments_minimum]
print(f"{len(l_video_ids)} videos with {n_comments_minimum}+ comments")

14 videos with 0+ comments


In [7]:
# Concurrent
comments_concurrent_results = await fetch_comment_threads(l_video_ids, comments_params) # Also works for 1 video ID as string

print(f"Fetched comments for {len(comments_concurrent_results)} videos")
print("Video ID    | Comments")
print("------------|------------")
for i in comments_concurrent_results.keys():
    print(f"{i} | {len(comments_concurrent_results[i])}")

Comments are disabled for video with params: {'part': 'id,replies,snippet', 'order': 'time', 'videoId': 'IyoGdwVIwWw'}
Fetched comments for 14 videos
Video ID    | Comments
------------|------------
7QEbjRKbrjY | 1799
f26kRRKrlj8 | 15
vdv05uU1m98 | 71
Zc57tP7ODNQ | 119
CG1bpu2KqeQ | 6
RDVXI84KSY8 | 85
PamZCTjqR0k | 23
IyoGdwVIwWw | 0
ipcpgND2-AY | 1
Bn0HOI9Qx0o | 43
8vwv23XYcrU | 11
0Hxf4Vf54PI | 3390
tC_orukfkGE | 25
HAnFA2FAl3o | 0


## Pipelines

In [8]:
# Search API -> Videos API -> CommentThreads API
svc_pipeline_results = await search_videos_comments(query, search_params, video_params, comments_params)

print(f"Data retrieved for {len(svc_pipeline_results)} videos")
print("Video ID    | Comments")
print("------------|------------")
for k,v in svc_pipeline_results.items():
    print(f"{k} | {len(v['commentThreads'])}")

15 videos found
15 videos with 0+ comments
Comments are disabled for video with params: {'part': 'id,replies,snippet', 'order': 'time', 'videoId': 'IyoGdwVIwWw'}
Data retrieved for 15 videos
Video ID    | Comments
------------|------------
7QEbjRKbrjY | 1799
5siCgOJXPdI | 52
f26kRRKrlj8 | 15
vdv05uU1m98 | 71
Zc57tP7ODNQ | 119
CG1bpu2KqeQ | 6
6A9gY1jRuEw | 238
PamZCTjqR0k | 23
IyoGdwVIwWw | 0
mMOBD9gUIVk | 12
ipcpgND2-AY | 1
8vwv23XYcrU | 11
0Hxf4Vf54PI | 3390
tC_orukfkGE | 25
HAnFA2FAl3o | 0
